# Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm
import os

## For Google Colab Users

This cell is for mounting your Google Drive to the Colab Notebook. If you are not using Google Colab, you can skip this cell

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Check for GPU in mac
# device = "mps" if torch.backends.mps.is_available() else "cpu"

device

# Data

## Transforming Data

In [ ]:
data_transforms = {

    'Training' : transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'Testing': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()

    ])
}

## Loading Data

In [ ]:
# directory: where training and testing data are
data_dir = '/kaggle/input/tumor-dataset/Dataset'
### START CODE HERE
# datasets.ImageFolder: (https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html)
# torch.utils.data.DataLoader: (https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

# image_datasets are dictionary of (type of dataset, dataloader)
# type of dataset are training and testing
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in data_transforms}

# DataLoader helps us for better performance and experience in data loading
torch.manual_seed(2098)
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in data_transforms}
### END CODE HERE

dataset_sizes = {x: len(image_datasets[x]) for x in data_transforms}
class_names = image_datasets['Training'].classes

dataset_sizes, class_names

## Samples of data

In [ ]:
samples, labels = next(iter(dataloaders['Testing']))
plt.figure(figsize=(17, 10))
plt.axis('off')
for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.imshow(samples[i].permute(1, 2, 0))
    plt.title(class_names[labels[i]])
    plt.axis('off')

# Model

## Loading Model

In [ ]:
# Loading are pretrained model in this task our model is resnet50 (https://www.youtube.com/watch?v=mGMpHyiN5lk)
### START CODE HERE

# Loading pretrained model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False
### END CODE HERE
model

## Preparing Model

In [ ]:
### START CODE HERE

# You have to change the (fc) layer of the model to compatible with your data
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=len(class_names))
# for param in model.fc.parameters():
#     param.requires_grad = True


### END CODE HERE
model = model.to(device)
model

# Training

## Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [ ]:
# you have to change it for better performance
optimizer = optim.Adam(model.parameters(), lr=0.01) # changed here

## Others

In [ ]:
# # # you can have other thongs like learning rate scheduler and ...
# from torch.optim.lr_scheduler import StepLR
# step_size = 6
# gamma = 0.85
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

## Train

In [ ]:
### START CODE HERE

losses = []
EPOCH = 15
# for training part you have to set model to train mode
model.train()

# loop on epochs
for e in tqdm(range(EPOCH)):

  # loop on batches
  for inputs, labels in dataloaders['Training']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # set the grad to zero
    optimizer.zero_grad()

    # forward part
    # hint: using of pytorch max method (https://pytorch.org/docs/stable/generated/torch.max.html)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    

    #  compute loss
    loss = criterion(outputs, labels)

    # backward part
    loss.backward()

    # update parameters
    optimizer.step()
  
  # added scheduler
#   scheduler.step()
  # you have to append loss for each epoch
  losses.append(loss.item())
### END CODE HERE

## Plot loss function

In [ ]:
# you have to calculate losses arrayin Train part
plt.plot(list(range(len(losses))), losses)
plt.show()

## Evaluate model

In [ ]:
### START CODE HERE

def calc_accuracy(data, model):
  corrects = 0

  # for testing part you have to set model to eval mode
  model.eval()
  for inputs, labels in tqdm(dataloaders[data]):
      inputs = inputs.to(device)
      labels = labels.to(device)

      with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)
        corrects += torch.sum(preds == labels.data)
  return corrects.double() / dataset_sizes[data]

### END CODE HERE

In [ ]:
# accuracy of training data
calc_accuracy('Training', model)

In [ ]:
# accuracy of testing data
calc_accuracy('Testing', model)

# Saving Model

In [ ]:
PATH = os.path.join('/kaggle/working/', 'model.ci')
torch.save(model, PATH)

# Loading and eval Model

In [ ]:
### START CODE HERE

model_for_eval = torch.load(PATH)
model_for_eval.to(device)

### END CODE HERE

In [ ]:
model_for_eval

In [ ]:
# accuracy of training data by loadded model
calc_accuracy('Training', model_for_eval)

In [ ]:
# accuracy of testing data by loadded model
calc_accuracy('Testing', model_for_eval)